In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from prophet import Prophet
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

c:\Users\quine\Documents\repos_github\Samsung_innovation_campus_proyecto\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('../data/03_casos/df_53_cleaned.csv')

In [3]:
df.head()

,ano,semana,ira_no_neumonia,neumonias_men5,neumonias_60mas,hospitalizados_men5,hospitalizados_60mas,defunciones_men5,defunciones_60mas,sub_reg_nt,...,defunciones_60mas_lag_3_semana,defunciones_60mas_lag_4_semana,defunciones_60mas_lag_5_semana,defunciones_60mas_lag_6_semana,mes,bimestre,trimestre,cuatrimestre,semestre,pandemia_covid
0,2000,1,2660,14,0,0,0,0,0,53,...,0.0,0.0,0.0,0.0,1,1,1,1,1,0
1,2000,2,2567,8,0,0,0,0,0,53,...,0.0,0.0,0.0,0.0,1,1,1,1,1,0
2,2000,3,2230,15,0,0,0,0,0,53,...,0.0,0.0,0.0,0.0,1,1,1,1,1,0
3,2000,4,2067,12,0,0,0,0,0,53,...,0.0,0.0,0.0,0.0,1,1,1,1,1,0
4,2000,5,1677,8,0,0,0,0,0,53,...,0.0,0.0,0.0,0.0,2,1,1,1,1,0


In [4]:
df.columns

Index(['ano', 'semana', 'ira_no_neumonia', 'neumonias_men5', 'neumonias_60mas',
       'hospitalizados_men5', 'hospitalizados_60mas', 'defunciones_men5',
       'defunciones_60mas', 'sub_reg_nt', 'ira_no_neumonia_lag_1_semana',
       'ira_no_neumonia_lag_2_semana', 'ira_no_neumonia_lag_3_semana',
       'ira_no_neumonia_lag_4_semana', 'ira_no_neumonia_lag_5_semana',
       'ira_no_neumonia_lag_6_semana', 'neumonias_men5_lag_1_semana',
       'neumonias_men5_lag_2_semana', 'neumonias_men5_lag_3_semana',
       'neumonias_men5_lag_4_semana', 'neumonias_men5_lag_5_semana',
       'neumonias_men5_lag_6_semana', 'neumonias_60mas_lag_1_semana',
       'neumonias_60mas_lag_2_semana', 'neumonias_60mas_lag_3_semana',
       'neumonias_60mas_lag_4_semana', 'neumonias_60mas_lag_5_semana',
       'neumonias_60mas_lag_6_semana', 'hospitalizados_men5_lag_1_semana',
       'hospitalizados_men5_lag_2_semana', 'hospitalizados_men5_lag_3_semana',
       'hospitalizados_men5_lag_4_semana', 'hospitalizad

Vamos a entrenar modelos de forecasting para la predicción de `hospitalizados_men5`, considerando los siguientes escenarios:

### Por años [2000 a 2023]

- Entrenar usando:  
  **Variables:** `ano`, `semana`  
  **Target:** `hospitalizados_men5`

- Entrenar usando:  
  **Variables:** `ano`, `semana`, `pandemia_covid`  
  **Target:** `hospitalizados_men5`

- Entrenar usando:  
  **Variables:** `ano`, `semana`, `ira_no_neumonia`, `neumonias_men5`, `neumonias_60mas`, `hospitalizados_60mas`, `defunciones_men5`, `defunciones_60mas`  
  **Target:** `hospitalizados_men5`

- Entrenar usando:  
  **Variables:** `ano`, `semana`, `ira_no_neumonia`, `neumonias_men5`, `neumonias_60mas`, `hospitalizados_60mas`, `defunciones_men5`, `defunciones_60mas`, y sus variables *lag* (`_lag_1_semana` a `_lag_6_semana`)  
  **Target:** `hospitalizados_men5`

- Entrenar usando:  
  **Variables:** `ano`, `semana`, `hospitalizados_60mas_lag_1_semana`, `defunciones_men5_lag_1_semana`, `defunciones_60mas_lag_1_semana`, `defunciones_men5_lag_3_semana`   
  **Target:** `hospitalizados_men5`


### Por años [2022 a 2023]

- Entrenar usando:  
  **Variables:** `ano`, `semana`  
  **Target:** `hospitalizados_men5`

- Entrenar usando:  
  **Variables:** `ano`, `semana`, `ira_no_neumonia`, `neumonias_men5`, `neumonias_60mas`, `hospitalizados_60mas`, `defunciones_men5`, `defunciones_60mas`  
  **Target:** `hospitalizados_men5`

- Entrenar usando:  
  **Variables:** `ano`, `semana`, `ira_no_neumonia`, `neumonias_men5`, `neumonias_60mas`, `hospitalizados_60mas`, `defunciones_men5`, `defunciones_60mas`, y sus variables *lag* (`_lag_1_semana` a `_lag_6_semana`)  
  **Target:** `hospitalizados_men5`

- Entrenar usando:  
  **Variables:** `ano`, `semana`, `hospitalizados_60mas_lag_1_semana`, `defunciones_men5_lag_1_semana`, `defunciones_60mas_lag_1_semana`, `defunciones_men5_lag_3_semana`   
  **Target:** `hospitalizados_men5`

## Modelos: Prophet, XGBoost y Random Forest

Entrenaremos tres tipos de modelos para comparar su desempeño:
- **Prophet**: Modelo de series temporales desarrollado por Meta, robusto a datos faltantes y cambios de tendencia
- **XGBoost**: Gradient Boosting de alto rendimiento basado en árboles de decisión
- **Random Forest**: Ensemble de árboles de decisión que reduce el overfitting

In [5]:
# Diccionario para almacenar resultados de todos los modelos
resultados_modelos = {}

In [6]:
# Definir conjuntos de variables según los criterios
variables_sets = {
    'set1_ano_semana': ['ano', 'semana'],
    'set2_con_pandemia': ['ano', 'semana', 'pandemia_covid'],
    'set3_principales': ['ano', 'semana', 'ira_no_neumonia', 'neumonias_men5', 'neumonias_60mas',
                         'hospitalizados_60mas', 'defunciones_men5', 'defunciones_60mas'],
    'set4_con_lags': ['ano', 'semana', 'ira_no_neumonia', 'neumonias_men5', 'neumonias_60mas',
                      'hospitalizados_60mas', 'defunciones_men5', 'defunciones_60mas'],
    'set5_lags_selectos': ['ano', 'semana', 'hospitalizados_60mas_lag_1_semana',
                           'defunciones_men5_lag_1_semana', 'defunciones_60mas_lag_1_semana',
                           'defunciones_men5_lag_3_semana']
}

# Agregar variables lag al set4
lag_vars = []
for var in ['ira_no_neumonia', 'neumonias_men5', 'neumonias_60mas',
            'hospitalizados_60mas', 'defunciones_men5', 'defunciones_60mas']:
    for i in range(1, 7):
        lag_vars.append(f'{var}_lag_{i}_semana')

variables_sets['set4_con_lags'].extend(lag_vars)

print("Conjuntos de variables definidos:")
for key, vars_list in variables_sets.items():
    print(f"\n{key}: {len(vars_list)} variables")
    print(f"  {', '.join(vars_list[:5])}{'...' if len(vars_list) > 5 else ''}")

Conjuntos de variables definidos:

set1_ano_semana: 2 variables
  ano, semana

set2_con_pandemia: 3 variables
  ano, semana, pandemia_covid

set3_principales: 8 variables
  ano, semana, ira_no_neumonia, neumonias_men5, neumonias_60mas...

set4_con_lags: 44 variables
  ano, semana, ira_no_neumonia, neumonias_men5, neumonias_60mas...

set5_lags_selectos: 6 variables
  ano, semana, hospitalizados_60mas_lag_1_semana, defunciones_men5_lag_1_semana, defunciones_60mas_lag_1_semana...


### Periodo 2000-2023 - Entrenamiento de Modelos

In [7]:
# Preparar datos para periodo 2000-2023
df_2000_2023 = df.copy().sort_values(['ano', 'semana']).reset_index(drop=True)
y_full = df_2000_2023['hospitalizados_men5']
split_idx = int(len(y_full) * 0.8)

print(f"Total observaciones 2000-2023: {len(y_full)}")
print(f"Train: {split_idx} observaciones")
print(f"Test: {len(y_full) - split_idx} observaciones")

Total observaciones 2000-2023: 1248
Train: 998 observaciones
Test: 250 observaciones


In [8]:
# Entrenar modelos para periodo 2000-2023
sets_2000_2023 = ['set1_ano_semana', 'set2_con_pandemia', 'set3_principales', 'set4_con_lags', 'set5_lags_selectos']

for set_name in sets_2000_2023:
    exog_vars = variables_sets[set_name]
    X = df_2000_2023[exog_vars]
    y = df_2000_2023['hospitalizados_men5']
    
    X_train = X[:split_idx]
    X_test = X[split_idx:]
    y_train = y[:split_idx]
    y_test = y[split_idx:]
    
    # Nombre descriptivo
    if set_name == 'set1_ano_semana':
        var_desc = 'ano, semana'
    elif set_name == 'set2_con_pandemia':
        var_desc = 'ano, semana, pandemia_covid'
    elif set_name == 'set3_principales':
        var_desc = 'principales'
    elif set_name == 'set4_con_lags':
        var_desc = 'principales + lags'
    else:
        var_desc = 'lags selectos'
    
    # ==================== Prophet ====================
    try:
        # Preparar datos en formato Prophet (requiere columnas 'ds' y 'y')
        df_prophet_train = pd.DataFrame({
            'ds': pd.to_datetime(df_2000_2023.loc[:split_idx-1, 'ano'].astype(str) + '-W' + 
                                 df_2000_2023.loc[:split_idx-1, 'semana'].astype(str) + '-1', 
                                 format='%Y-W%W-%w'),
            'y': y_train.values
        })
        
        # Agregar regresores adicionales (excluyendo ano y semana)
        regressors = [col for col in exog_vars if col not in ['ano', 'semana']]
        for reg in regressors:
            df_prophet_train[reg] = X_train[reg].values
        
        # Entrenar Prophet
        model_prophet = Prophet(yearly_seasonality=True, weekly_seasonality=False, 
                                daily_seasonality=False, seasonality_mode='multiplicative')
        
        for reg in regressors:
            model_prophet.add_regressor(reg)
        
        model_prophet.fit(df_prophet_train)
        
        # Preparar datos de prueba
        df_prophet_test = pd.DataFrame({
            'ds': pd.to_datetime(df_2000_2023.loc[split_idx:, 'ano'].astype(str) + '-W' + 
                                 df_2000_2023.loc[split_idx:, 'semana'].astype(str) + '-1', 
                                 format='%Y-W%W-%w')
        })
        
        for reg in regressors:
            df_prophet_test[reg] = X_test[reg].values
        
        # Predicciones
        pred_prophet = model_prophet.predict(df_prophet_test)
        pred_prophet_values = pred_prophet['yhat'].values
        
        resultados_modelos[f'Prophet_2000-2023_{set_name}'] = {
            'modelo': 'Prophet',
            'periodo': '2000-2023',
            'variables': var_desc,
            'MAE': mean_absolute_error(y_test, pred_prophet_values),
            'RMSE': np.sqrt(mean_squared_error(y_test, pred_prophet_values)),
            'R2': r2_score(y_test, pred_prophet_values)
        }
        print(f"✓ Prophet completado: {set_name}")
    except Exception as e:
        print(f"✗ Error en Prophet {set_name}: {e}")
    
    # ==================== XGBoost ====================
    try:
        model_xgboost = XGBRegressor(
            n_estimators=500,
            learning_rate=0.05,
            max_depth=6,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42,
            verbosity=0
        )
        model_xgboost.fit(X_train, y_train)
        pred_xgboost = model_xgboost.predict(X_test)
        
        resultados_modelos[f'XGBoost_2000-2023_{set_name}'] = {
            'modelo': 'XGBoost',
            'periodo': '2000-2023',
            'variables': var_desc,
            'MAE': mean_absolute_error(y_test, pred_xgboost),
            'RMSE': np.sqrt(mean_squared_error(y_test, pred_xgboost)),
            'R2': r2_score(y_test, pred_xgboost)
        }
        print(f"✓ XGBoost completado: {set_name}")
    except Exception as e:
        print(f"✗ Error en XGBoost {set_name}: {e}")
    
    # ==================== Random Forest ====================
    try:
        model_rf = RandomForestRegressor(
            n_estimators=500,
            max_depth=10,
            min_samples_split=5,
            min_samples_leaf=2,
            max_features='sqrt',
            random_state=42,
            n_jobs=-1,
            verbose=0
        )
        model_rf.fit(X_train, y_train)
        pred_rf = model_rf.predict(X_test)
        
        resultados_modelos[f'RandomForest_2000-2023_{set_name}'] = {
            'modelo': 'RandomForest',
            'periodo': '2000-2023',
            'variables': var_desc,
            'MAE': mean_absolute_error(y_test, pred_rf),
            'RMSE': np.sqrt(mean_squared_error(y_test, pred_rf)),
            'R2': r2_score(y_test, pred_rf)
        }
        print(f"✓ Random Forest completado: {set_name}")
    except Exception as e:
        print(f"✗ Error en Random Forest {set_name}: {e}")
    
    print(f"Completado conjunto: {set_name} para periodo 2000-2023\n")

print(f"\n{'='*80}")
print(f"Total de modelos entrenados (2000-2023): {len(resultados_modelos)}")
print(f"{'='*80}")

16:02:42 - cmdstanpy - INFO - Chain [1] start processing
16:02:43 - cmdstanpy - INFO - Chain [1] done processing


✓ Prophet completado: set1_ano_semana
✓ XGBoost completado: set1_ano_semana


16:02:45 - cmdstanpy - INFO - Chain [1] start processing


✓ Random Forest completado: set1_ano_semana
Completado conjunto: set1_ano_semana para periodo 2000-2023



16:02:45 - cmdstanpy - INFO - Chain [1] done processing


...

✓ Prophet completado: set4_con_lags
✓ XGBoost completado: set4_con_lags


16:02:51 - cmdstanpy - INFO - Chain [1] start processing


✓ Random Forest completado: set4_con_lags
Completado conjunto: set4_con_lags para periodo 2000-2023



16:02:51 - cmdstanpy - INFO - Chain [1] done processing


✓ Prophet completado: set5_lags_selectos
✓ XGBoost completado: set5_lags_selectos
✓ Random Forest completado: set5_lags_selectos
Completado conjunto: set5_lags_selectos para periodo 2000-2023


Total de modelos entrenados (2000-2023): 15


### Periodo 2022-2023 - Entrenamiento de Modelos

In [9]:
# Preparar datos para periodo 2022-2023
df_2022_2023 = df[df['ano'] >= 2022].copy().sort_values(['ano', 'semana']).reset_index(drop=True)
y_full_2022 = df_2022_2023['hospitalizados_men5']
split_idx_2022 = int(len(y_full_2022) * 0.8)

print(f"Total observaciones 2022-2023: {len(y_full_2022)}")
print(f"Train: {split_idx_2022} observaciones")
print(f"Test: {len(y_full_2022) - split_idx_2022} observaciones")

Total observaciones 2022-2023: 104
Train: 83 observaciones
Test: 21 observaciones


In [10]:
# Entrenar modelos para periodo 2022-2023
sets_2022_2023 = ['set1_ano_semana', 'set3_principales', 'set4_con_lags', 'set5_lags_selectos']

for set_name in sets_2022_2023:
    exog_vars = variables_sets[set_name]
    X = df_2022_2023[exog_vars]
    y = df_2022_2023['hospitalizados_men5']
    
    X_train = X[:split_idx_2022]
    X_test = X[split_idx_2022:]
    y_train = y[:split_idx_2022]
    y_test = y[split_idx_2022:]
    
    # Nombre descriptivo
    if set_name == 'set1_ano_semana':
        var_desc = 'ano, semana'
    elif set_name == 'set3_principales':
        var_desc = 'principales'
    elif set_name == 'set4_con_lags':
        var_desc = 'principales + lags'
    else:
        var_desc = 'lags selectos'
    
    # ==================== Prophet ====================
    try:
        # Preparar datos en formato Prophet
        df_prophet_train = pd.DataFrame({
            'ds': pd.to_datetime(df_2022_2023.loc[:split_idx_2022-1, 'ano'].astype(str) + '-W' + 
                                 df_2022_2023.loc[:split_idx_2022-1, 'semana'].astype(str) + '-1', 
                                 format='%Y-W%W-%w'),
            'y': y_train.values
        })
        
        # Agregar regresores adicionales
        regressors = [col for col in exog_vars if col not in ['ano', 'semana']]
        for reg in regressors:
            df_prophet_train[reg] = X_train[reg].values
        
        # Entrenar Prophet
        model_prophet = Prophet(yearly_seasonality=True, weekly_seasonality=False, 
                                daily_seasonality=False, seasonality_mode='multiplicative')
        
        for reg in regressors:
            model_prophet.add_regressor(reg)
        
        model_prophet.fit(df_prophet_train)
        
        # Preparar datos de prueba
        df_prophet_test = pd.DataFrame({
            'ds': pd.to_datetime(df_2022_2023.loc[split_idx_2022:, 'ano'].astype(str) + '-W' + 
                                 df_2022_2023.loc[split_idx_2022:, 'semana'].astype(str) + '-1', 
                                 format='%Y-W%W-%w')
        })
        
        for reg in regressors:
            df_prophet_test[reg] = X_test[reg].values
        
        # Predicciones
        pred_prophet = model_prophet.predict(df_prophet_test)
        pred_prophet_values = pred_prophet['yhat'].values
        
        resultados_modelos[f'Prophet_2022-2023_{set_name}'] = {
            'modelo': 'Prophet',
            'periodo': '2022-2023',
            'variables': var_desc,
            'MAE': mean_absolute_error(y_test, pred_prophet_values),
            'RMSE': np.sqrt(mean_squared_error(y_test, pred_prophet_values)),
            'R2': r2_score(y_test, pred_prophet_values)
        }
        print(f"✓ Prophet completado: {set_name}")
    except Exception as e:
        print(f"✗ Error en Prophet {set_name}: {e}")
    
    # ==================== XGBoost ====================
    try:
        model_xgboost = XGBRegressor(
            n_estimators=500,
            learning_rate=0.05,
            max_depth=6,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42,
            verbosity=0
        )
        model_xgboost.fit(X_train, y_train)
        pred_xgboost = model_xgboost.predict(X_test)
        
        resultados_modelos[f'XGBoost_2022-2023_{set_name}'] = {
            'modelo': 'XGBoost',
            'periodo': '2022-2023',
            'variables': var_desc,
            'MAE': mean_absolute_error(y_test, pred_xgboost),
            'RMSE': np.sqrt(mean_squared_error(y_test, pred_xgboost)),
            'R2': r2_score(y_test, pred_xgboost)
        }
        print(f"✓ XGBoost completado: {set_name}")
    except Exception as e:
        print(f"✗ Error en XGBoost {set_name}: {e}")
    
    # ==================== Random Forest ====================
    try:
        model_rf = RandomForestRegressor(
            n_estimators=500,
            max_depth=10,
            min_samples_split=5,
            min_samples_leaf=2,
            max_features='sqrt',
            random_state=42,
            n_jobs=-1,
            verbose=0
        )
        model_rf.fit(X_train, y_train)
        pred_rf = model_rf.predict(X_test)
        
        resultados_modelos[f'RandomForest_2022-2023_{set_name}'] = {
            'modelo': 'RandomForest',
            'periodo': '2022-2023',
            'variables': var_desc,
            'MAE': mean_absolute_error(y_test, pred_rf),
            'RMSE': np.sqrt(mean_squared_error(y_test, pred_rf)),
            'R2': r2_score(y_test, pred_rf)
        }
        print(f"✓ Random Forest completado: {set_name}")
    except Exception as e:
        print(f"✗ Error en Random Forest {set_name}: {e}")
    
    print(f"Completado conjunto: {set_name} para periodo 2022-2023\n")

print(f"\n{'='*80}")
print(f"Total de modelos entrenados (todos): {len(resultados_modelos)}")
print(f"{'='*80}")

16:02:53 - cmdstanpy - INFO - Chain [1] start processing
16:02:53 - cmdstanpy - INFO - Chain [1] done processing


✓ Prophet completado: set1_ano_semana
✓ XGBoost completado: set1_ano_semana


16:02:54 - cmdstanpy - INFO - Chain [1] start processing


✓ Random Forest completado: set1_ano_semana
Completado conjunto: set1_ano_semana para periodo 2022-2023



16:02:54 - cmdstanpy - INFO - Chain [1] done processing


...

✓ Prophet completado: set4_con_lags
✓ XGBoost completado: set4_con_lags


16:02:57 - cmdstanpy - INFO - Chain [1] start processing


✓ Random Forest completado: set4_con_lags
Completado conjunto: set4_con_lags para periodo 2022-2023



16:02:58 - cmdstanpy - INFO - Chain [1] done processing


✓ Prophet completado: set5_lags_selectos
✓ XGBoost completado: set5_lags_selectos
✓ Random Forest completado: set5_lags_selectos
Completado conjunto: set5_lags_selectos para periodo 2022-2023


Total de modelos entrenados (todos): 27


---

## Tabla Comparativa Completa - Formato Pivot Table

In [11]:
# Crear DataFrame con todos los resultados
df_resultados = pd.DataFrame(resultados_modelos).T
df_resultados = df_resultados.reset_index()
df_resultados.rename(columns={'index': 'nombre_completo'}, inplace=True)

# Redondear métricas
df_resultados[['MAE', 'RMSE', 'R2']] = df_resultados[['MAE', 'RMSE', 'R2']].round(4)

# Crear tabla pivote
df_pivot = df_resultados[['periodo', 'modelo', 'variables', 'MAE', 'RMSE', 'R2']].copy()
df_pivot = df_pivot.sort_values(['periodo', 'modelo', 'variables']).reset_index(drop=True)

print("="*100)
print("TABLA COMPARATIVA - MODELOS Prophet, XGBoost y Random Forest")
print("="*100)
print("\nTarget: hospitalizados_men5")
print("\nColumnas:")
print("  - Periodo: Rango de años utilizado")
print("  - Modelo: Prophet, XGBoost o Random Forest")
print("  - Variables: Conjunto de variables utilizadas")
print("  - MAE: Mean Absolute Error (menor es mejor)")
print("  - RMSE: Root Mean Squared Error (menor es mejor)")
print("  - R²: Coeficiente de Determinación (mayor es mejor, cercano a 1)")
print("\n" + "="*100)
print(df_pivot.to_string(index=True))
print("\n" + "="*100)

TABLA COMPARATIVA - MODELOS Prophet, XGBoost y Random Forest

Target: hospitalizados_men5

Columnas:
  - Periodo: Rango de años utilizado
  - Modelo: Prophet, XGBoost o Random Forest
  - Variables: Conjunto de variables utilizadas
  - MAE: Mean Absolute Error (menor es mejor)
  - RMSE: Root Mean Squared Error (menor es mejor)
  - R²: Coeficiente de Determinación (mayor es mejor, cercano a 1)

      periodo        modelo                    variables        MAE       RMSE         R2
0   2000-2023       Prophet                  ano, semana   3.080952    3.91489  -0.039994
1   2000-2023       Prophet  ano, semana, pandemia_covid   3.145702   3.899254  -0.031703
2   2000-2023       Prophet                lags selectos   3.326036    4.25055  -0.225976
3   2000-2023       Prophet                  principales   3.088931   3.970884  -0.069956
4   2000-2023       Prophet           principales + lags   3.310446   4.275208  -0.240241
5   2000-2023  RandomForest                  ano, semana   3.510

### Análisis de Mejores Modelos

In [12]:
print("="*100)
print("ANÁLISIS DE MEJORES MODELOS")
print("="*100)

# Mejor modelo global
print("\n1. MEJOR MODELO GLOBAL:")
print("-" * 100)
mejor_mae = df_pivot.loc[df_pivot['MAE'].idxmin()]
mejor_rmse = df_pivot.loc[df_pivot['RMSE'].idxmin()]
mejor_r2 = df_pivot.loc[df_pivot['R2'].idxmax()]

print(f"\nMenor MAE: {mejor_mae['MAE']:.4f}")
print(f"  Modelo: {mejor_mae['modelo']}")
print(f"  Periodo: {mejor_mae['periodo']}")
print(f"  Variables: {mejor_mae['variables']}")

print(f"\nMenor RMSE: {mejor_rmse['RMSE']:.4f}")
print(f"  Modelo: {mejor_rmse['modelo']}")
print(f"  Periodo: {mejor_rmse['periodo']}")
print(f"  Variables: {mejor_rmse['variables']}")

print(f"\nMayor R²: {mejor_r2['R2']:.4f}")
print(f"  Modelo: {mejor_r2['modelo']}")
print(f"  Periodo: {mejor_r2['periodo']}")
print(f"  Variables: {mejor_r2['variables']}")

# Mejor modelo por periodo
print("\n" + "="*100)
print("2. MEJOR MODELO POR PERIODO:")
print("-" * 100)
for periodo in df_pivot['periodo'].unique():
    df_periodo = df_pivot[df_pivot['periodo'] == periodo]
    mejor = df_periodo.loc[df_periodo['R2'].idxmax()]
    
    print(f"\n{periodo}:")
    print(f"  Modelo: {mejor['modelo']}")
    print(f"  Variables: {mejor['variables']}")
    print(f"  MAE: {mejor['MAE']:.4f}")
    print(f"  RMSE: {mejor['RMSE']:.4f}")
    print(f"  R²: {mejor['R2']:.4f}")

# Mejor modelo por tipo
print("\n" + "="*100)
print("3. MEJOR CONFIGURACIÓN POR TIPO DE MODELO:")
print("-" * 100)
for modelo_tipo in df_pivot['modelo'].unique():
    df_modelo = df_pivot[df_pivot['modelo'] == modelo_tipo]
    mejor = df_modelo.loc[df_modelo['R2'].idxmax()]
    
    print(f"\n{modelo_tipo}:")
    print(f"  Periodo: {mejor['periodo']}")
    print(f"  Variables: {mejor['variables']}")
    print(f"  MAE: {mejor['MAE']:.4f}")
    print(f"  RMSE: {mejor['RMSE']:.4f}")
    print(f"  R²: {mejor['R2']:.4f}")

# Desempeño promedio por conjunto de variables
print("\n" + "="*100)
print("4. DESEMPEÑO PROMEDIO POR CONJUNTO DE VARIABLES:")
print("-" * 100)
agg_vars = df_pivot.groupby('variables')[['MAE', 'RMSE', 'R2']].mean().round(4)
agg_vars = agg_vars.sort_values('R2', ascending=False)
print(agg_vars)

print("\n" + "="*100)

ANÁLISIS DE MEJORES MODELOS

1. MEJOR MODELO GLOBAL:
----------------------------------------------------------------------------------------------------

Menor MAE: 1.8773
  Modelo: RandomForest
  Periodo: 2022-2023
  Variables: principales + lags

Menor RMSE: 2.3162
  Modelo: RandomForest
  Periodo: 2022-2023
  Variables: principales + lags

Mayor R²: 0.2647
  Modelo: RandomForest
  Periodo: 2000-2023
  Variables: principales

2. MEJOR MODELO POR PERIODO:
----------------------------------------------------------------------------------------------------

2000-2023:
  Modelo: RandomForest
  Variables: principales
  MAE: 2.7586
  RMSE: 3.2917
  R²: 0.2647

2022-2023:
  Modelo: RandomForest
  Variables: principales + lags
  MAE: 1.8773
  RMSE: 2.3162
  R²: -0.3232

3. MEJOR CONFIGURACIÓN POR TIPO DE MODELO:
----------------------------------------------------------------------------------------------------

Prophet:
  Periodo: 2000-2023
  Variables: ano, semana, pandemia_covid
  MAE: 3

### Visualización de Resultados

### Exportar Resultados

In [13]:
# Guardar resultados en CSV
output_path = '../data/resultados_modelos_prophet_xgboost_randomforest.csv'
df_pivot.to_csv(output_path, index=False)
print(f"Resultados guardados en: {output_path}")

# Resumen final
print("\n" + "="*100)
print("RESUMEN FINAL")
print("="*100)
print(f"\nTotal de modelos entrenados: {len(df_pivot)}")
print(f"Periodos analizados: {df_pivot['periodo'].nunique()}")
print(f"Tipos de modelos: {', '.join(df_pivot['modelo'].unique())}")
print(f"Conjuntos de variables: {df_pivot['variables'].nunique()}")
print("\nModelos:")
print("  - Prophet: Series temporales con tendencias y estacionalidad")
print("  - XGBoost: Gradient Boosting de alto rendimiento")
print("  - Random Forest: Ensemble de árboles de decisión")
print("\nArchivo de resultados generado exitosamente.")
print("="*100)

Resultados guardados en: ../data/resultados_modelos_prophet_xgboost_randomforest.csv

RESUMEN FINAL

Total de modelos entrenados: 27
Periodos analizados: 2
Tipos de modelos: Prophet, RandomForest, XGBoost
Conjuntos de variables: 5

Modelos:
  - Prophet: Series temporales con tendencias y estacionalidad
  - XGBoost: Gradient Boosting de alto rendimiento
  - Random Forest: Ensemble de árboles de decisión

Archivo de resultados generado exitosamente.
